In [1]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
    sys.path.insert(0, module_path)
# endregion       #
# --------------- #

In [9]:
%autoreload 2
from examples.ft.experiments import eval_model
from examples.ft.utils import animate

results = eval_model(
    'memories',
    model_path='../models/checkpoints/model_iRainbowModel_0.0_5.pkl',
    config_path='../config.yaml',
    n_games=1000
)

In [17]:
%autoreload 2

results['memories'].save('../models/checkpoints/memories.pkl')

In [7]:
%autoreload 2

from examples.ft.models.inverse import MLPBCModel
from examples.ft.models.buffer import ActionBatchReplayBuffer as Buffer

memories = Buffer.load('../models/checkpoints/memories.pkl')
model = MLPBCModel(
    state_size=[7, 11, 11],
    action_size=5,
    layer_size=250,
    epsilon=0.,
    n_channels=7,
    frames=5,
    seed=1,
    device='cpu',
    memory=memories,
    LR=.001
)

model.train_model()

61241.886790114455